In [1]:
%matplotlib inline

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [2]:
ixy = xr.DataArray([1,1,1,2,2,2,3,3,3,4,4,4], dims='pft')
lon = xr.DataArray([0,0,0,110,110,110,212,212,212,358,358,358], dims='pft')
jxy = xr.DataArray([1,1,1,1,2,2,2,2,3,3,3,3], dims='pft')
lat = xr.DataArray([-90,-90,-90,-90,10,10,10,10,84,84,84,84], dims='pft')
active = xr.DataArray([1,1,0,1,0,0,0,0,1,0,1,1], dims='pft')
wtlunit = xr.DataArray([0,0.1,0,0,0.2,0.3,1,0,0,0.1,0.2,0], dims='pft')
itype_lunit = xr.DataArray([1,1,2,7,4,5,6,7,8,9,2,4], dims='pft')
tsa = xr.DataArray([3,7,9,0,12,4,5,6,8,1,2,90], dims='pft')

In [9]:
# to turn into numpy arrays
#lon = lon.values
#ixy = ixy.values

<xarray.DataArray (pft: 12)>
array([1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4])
Dimensions without coordinates: pft

In [10]:
# pull out values (prolly don't use this one cause it doesn't retain the shape and size of the array )
lon[ixy==2]

<xarray.DataArray (pft: 3)>
array([110, 110, 110])
Dimensions without coordinates: pft

In [11]:
# mask values (preferred for now)
lon.where(ixy==2)

<xarray.DataArray (pft: 12)>
array([ nan,  nan,  nan, 110., 110., 110.,  nan,  nan,  nan,  nan,  nan,  nan])
Dimensions without coordinates: pft

In [7]:
# count unique values per lat/lon pair 
max_length = 0
for i in range(1,5):
    for j in range(1,4):
        print(i,j)
        print(lon.where(ixy==i).where(jxy==j).values)
        new_length = tsa.where(ixy==i).where(jxy==j).dropna(dim='pft').values.size
        if new_length > max_length:
            max_length = new_length

print('Max number of unique values per unique lat/lon pair =',max_length)

1 1
[ 0.  0.  0. nan nan nan nan nan nan nan nan nan]
1 2
[nan nan nan nan nan nan nan nan nan nan nan nan]
1 3
[nan nan nan nan nan nan nan nan nan nan nan nan]
2 1
[ nan  nan  nan 110.  nan  nan  nan  nan  nan  nan  nan  nan]
2 2
[ nan  nan  nan  nan 110. 110.  nan  nan  nan  nan  nan  nan]
2 3
[nan nan nan nan nan nan nan nan nan nan nan nan]
3 1
[nan nan nan nan nan nan nan nan nan nan nan nan]
3 2
[ nan  nan  nan  nan  nan  nan 212. 212.  nan  nan  nan  nan]
3 3
[ nan  nan  nan  nan  nan  nan  nan  nan 212.  nan  nan  nan]
4 1
[nan nan nan nan nan nan nan nan nan nan nan nan]
4 2
[nan nan nan nan nan nan nan nan nan nan nan nan]
4 3
[ nan  nan  nan  nan  nan  nan  nan  nan  nan 358. 358. 358.]
Max number of unique values per unique lat/lon pair = 3


In [8]:
# find max pft for each unique lat/lon/lunit combo
max_length = 0
tsa = tsa.where(itype_lunit==1) # only look at longitude values where the lunit type is 1 aka veg
# ^ this is the first prerequisite 

# now loop through each latitude/longitude combo 
for i in range(1,5):
    for j in range(1,4):
        print(i,j)
        print(tsa.where(ixy==i).where(jxy==j).values)
        new_length = tsa.where(ixy==i).where(jxy==j).dropna(dim='pft').values.size
        if new_length > max_length:
            max_length = new_length

print('Max unique values per lat/lon pair =', max_length)

1 1
[ 3.  7. nan nan nan nan nan nan nan nan nan nan]
1 2
[nan nan nan nan nan nan nan nan nan nan nan nan]
1 3
[nan nan nan nan nan nan nan nan nan nan nan nan]
2 1
[nan nan nan nan nan nan nan nan nan nan nan nan]
2 2
[nan nan nan nan nan nan nan nan nan nan nan nan]
2 3
[nan nan nan nan nan nan nan nan nan nan nan nan]
3 1
[nan nan nan nan nan nan nan nan nan nan nan nan]
3 2
[nan nan nan nan nan nan nan nan nan nan nan nan]
3 3
[nan nan nan nan nan nan nan nan nan nan nan nan]
4 1
[nan nan nan nan nan nan nan nan nan nan nan nan]
4 2
[nan nan nan nan nan nan nan nan nan nan nan nan]
4 3
[nan nan nan nan nan nan nan nan nan nan nan nan]
Max unique values per lat/lon pair = 2


In [ ]:
# Game Plan 
# Go through and create list of tsa values for each unique lat/lon pair for the land unit of interest 
# Then initialize a cube with dimensions of lat x lon x max pft (for that land unit?)
# Loop through the tsa list and fill in each value one by one into the previously created cube 

In [3]:
cube1 = np.full((4,3,3), float("NaN"))

In [4]:
for i in range(1,5):
    for j in range(1,4):
        print(i,j)

1 1
1 2
1 3
2 1
2 2
2 3
3 1
3 2
3 3
4 1
4 2
4 3


In [5]:
jxy

<xarray.DataArray (pft: 12)>
array([1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3])
Dimensions without coordinates: pft

In [12]:
# DOUBLE CHECK OUTPUT 
# first mask for only the desired land unit type 
tsa = xr.DataArray([3,7,9,0,12,4,5,6,8,1,2,90], dims='pft')
for i in range(1,5):
    for j in range(1,4):
        print('lat/lon indices:',i, ',', j) #
        v = tsa.where(ixy==i).where(jxy==j).dropna(dim='pft').values
        print('tsa values at lat/lon index', v) #
        print('number of tsa values at that lat/lon:', len(v)) #
        for k in range(len(v)):
            print('cube dims: i,j,k =',i,j,k) # 
            cube1[i-1,j-1,k] = v[k]

lat/lon indices: 1 , 1
tsa values at lat/lon index [3. 7. 9.]
number of tsa values at that lat/lon: 3
cube dims: i,j,k = 1 1 0
cube dims: i,j,k = 1 1 1
cube dims: i,j,k = 1 1 2
lat/lon indices: 1 , 2
tsa values at lat/lon index []
number of tsa values at that lat/lon: 0
lat/lon indices: 1 , 3
tsa values at lat/lon index []
number of tsa values at that lat/lon: 0
lat/lon indices: 2 , 1
tsa values at lat/lon index [0.]
number of tsa values at that lat/lon: 1
cube dims: i,j,k = 2 1 0
lat/lon indices: 2 , 2
tsa values at lat/lon index [12.  4.]
number of tsa values at that lat/lon: 2
cube dims: i,j,k = 2 2 0
cube dims: i,j,k = 2 2 1
lat/lon indices: 2 , 3
tsa values at lat/lon index []
number of tsa values at that lat/lon: 0
lat/lon indices: 3 , 1
tsa values at lat/lon index []
number of tsa values at that lat/lon: 0
lat/lon indices: 3 , 2
tsa values at lat/lon index [5. 6.]
number of tsa values at that lat/lon: 2
cube dims: i,j,k = 3 2 0
cube dims: i,j,k = 3 2 1
lat/lon indices: 3 , 3
tsa

In [13]:
cube1

array([[[ 3.,  7.,  9.],
        [ 0., 12., nan],
        [nan,  5.,  8.]],

       [[ 0., nan, nan],
        [12.,  4., nan],
        [ 5.,  6., nan]],

       [[ 9., nan, nan],
        [ 5.,  6., nan],
        [ 8., nan, nan]],

       [[nan, nan, nan],
        [nan, nan, nan],
        [ 1.,  2., 90.]]])

In [17]:
cube1[3,2,0]

1.0

In [44]:
cube1

array([[[ 0.,  0., nan],
        [nan, nan, nan],
        [nan, nan, nan]],

       [[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]],

       [[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]],

       [[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]]])

In [48]:
cube2 = np.full((4,3,3), float("NaN"))
list = []
v = tsa.where(ixy==1).where(active==1).dropna(dim='pft').values
for i in range(len(v)):
    cube2[0,0,i] = v[i]

In [49]:
cube2

array([[[ 3.,  7., nan],
        [nan, nan, nan],
        [nan, nan, nan]],

       [[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]],

       [[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]],

       [[nan, nan, nan],
        [nan, nan, nan],
        [nan, nan, nan]]])

Old Code

In [61]:
# Set up empty data cubes 
#cube = np.empty((4,3))
cube = []
for i in range(60):
    

In [62]:
cube.shape

(4, 3)

In [63]:
arr = []
for i in range(10):
    a = np.random.rand(3,3)
    arr.append(a)

np.array(arr).shape
# (10, 3, 3)

(10, 3, 3)

In [9]:
cube.shape[0]

4

In [5]:
i,j = 1,1
list = [1]
for r in range(len(ixy)):
    if ixy[r] == i:
        list[0] = float(wtlunit[r].values)
        list.append(float(wtlunit[r].values))
list

[0.0, 0.0, 0.1, 0.0]

In [33]:
type(list[1])

numpy.ndarray

In [15]:
for i in range(cube.shape[0]):
    for j in range(cube.shape[1]):
        cube[i,j] = i+j

In [16]:
cube

array([[0., 1., 2.],
       [1., 2., 3.],
       [2., 3., 4.],
       [3., 4., 5.]])

In [39]:
cube[0,2]

2.0